In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import joblib

In [37]:
df = pd.read_csv(r"E:\APythonSoftware_2025\MP4\swiggy.csv")
df.head()

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [38]:
df["rating"] = df["rating"].replace("--",np.nan)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [39]:
df.dropna(inplace=True)

In [40]:
df.drop(df[df["cuisine"] == "8:15 To 11:30 Pm"].index, inplace=True)

In [41]:
df.shape

(61343, 11)

In [42]:
df["rating_count"] = df["rating_count"].replace('Too Few Ratings','12')

df["rating_count"] = df["rating_count"].str.replace("+ ratings","",regex=False)

df["rating_count"] = df["rating_count"].str.replace("K","000",regex=False)

df["rating_count"] = pd.to_numeric(df["rating_count"], errors='coerce') #--> Converting the datatype

In [43]:
df.drop(df[df["rating_count"] < 21 ].index, inplace=True)

In [44]:
df.shape

(39739, 11)

In [45]:
df['rating_count'].value_counts()

rating_count
100      20514
50       11985
500       4390
1000      2737
5000        98
10000       15
Name: count, dtype: int64

In [46]:
df[df.duplicated()]

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu


In [47]:
df.isnull().sum()

id              0
name            0
city            0
rating          0
rating_count    0
cost            0
cuisine         0
lic_no          0
link            0
address         0
menu            0
dtype: int64

In [48]:
df.drop(["id","lic_no","link","menu","address"],axis="columns",inplace=True)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39739 entries, 1 to 148509
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          39739 non-null  object 
 1   city          39739 non-null  object 
 2   rating        39739 non-null  float64
 3   rating_count  39739 non-null  int64  
 4   cost          39739 non-null  object 
 5   cuisine       39739 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.1+ MB


In [50]:
df.dropna(subset = ["cost"],inplace = True) #--> Removing the Null values

df["cost"] = df["cost"].str.replace("₹", "", regex=False)

df["cost"] = pd.to_numeric(df["cost"], errors="coerce") #--> Converting the datatype

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39739 entries, 1 to 148509
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          39739 non-null  object 
 1   city          39739 non-null  object 
 2   rating        39739 non-null  float64
 3   rating_count  39739 non-null  int64  
 4   cost          39739 non-null  int64  
 5   cuisine       39739 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.1+ MB


In [52]:
# Function to extract value after the last comma inside quotes
def extract_city_name(city):
    if pd.isna(city):
        return city  # Skip NaNs
    city = city.strip()

    # Remove outer quotes if present
    if (city.startswith('"') and city.endswith('"')) or (city.startswith("'") and city.endswith("'")):
        city = city[1:-1]

    # Extract part after the last comma, or return as is
    if ',' in city:
        return city.split(',')[-1].strip()
    else:
        return city.strip()


In [53]:
# Apply function to the 'city' column
df['city_cleaned'] = df['city'].apply(extract_city_name)

df.drop("city",axis="columns", inplace=True)

# Check the result
print(df['city_cleaned'].unique())

['Abohar' 'Adilabad' 'Adityapur' 'Adoni' 'Agartala' 'Agra' 'Ahmedabad'
 'Ahmednagar' 'Ajmer' 'Akola' 'Alappuzha' 'Aligarh' 'Alipurduar'
 'Allahabad' 'Alwar' 'Ambala' 'Ambikapur' 'Ambur' 'Amravati' 'Amreli'
 'Amritsar' 'Anand' 'Anantapur' 'Ankleshwar' 'Arambagh' 'Arrah' 'Asansol'
 'Aurangabad' 'Aurangabad_bihar' 'Azamgarh' 'Baddi' 'Bagalkot' 'Bagdogra'
 'Bahadurgarh' 'Bahraich' 'Balangir' 'Balasore' 'Ballari' 'Balrampur'
 'Balurghat' 'Banda' 'Bangalore' 'Bantwal' 'Bapatlachirala' 'Baramati'
 'Baran' 'Bardhaman' 'Bardoli' 'Bareilly' 'Barmer' 'Barnala' 'Barshi'
 'Barwani' 'Basirhat' 'Basti' 'Batala' 'Bathinda' 'Beawar' 'Beed'
 'Begusarai' 'Bela-pratapgarh' 'Belgaum' 'Berhampore' 'Berhampur'
 'Bettiah' 'Betul' 'Bhadohi' 'Bhadrachalam' 'Bhagalpur' 'Bhandara'
 'Bharabanki' 'Bharatpur' 'Bharuch' 'Bhatkal' 'Bhavnagar' 'Bhawanipatna'
 'Bhilai' 'Bhilwara' 'Bhimavaram' 'Bhind' 'Bhiwadi' 'Bhiwani' 'Bhopal'
 'Bhubaneswar' 'Bhuj' 'Bhusawal' 'Bidar' 'Biharsharif' 'Bijapur' 'Bijnor'
 'Bikaner' 'Bilasp

In [54]:
df.columns

Index(['name', 'rating', 'rating_count', 'cost', 'cuisine', 'city_cleaned'], dtype='object')

In [55]:
df.to_csv("cleaned_data.csv",index=False)

In [56]:
df.shape

(39739, 6)

In [ ]:
# Data encoding 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Read your CSV file
cleaned_df = df[['name','city_cleaned', 'cuisine', 'rating', 'rating_count', 'cost']].copy()

# Label Encoding
label = LabelEncoder()
cleaned_df["name_encoded"] = label.fit_transform(cleaned_df["name"])

# One-hot encoder
categorical_cols = ['city_cleaned', 'cuisine']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(cleaned_df[categorical_cols])

# DataFrame of encoded features
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols))

# Adding back numerical columns including name_encoded
numerical_df = cleaned_df[['name_encoded', 'rating', 'rating_count', 'cost']].reset_index(drop=True)
combined_df = pd.concat([encoded_df, numerical_df], axis=1)

combined_df.to_csv("encoded_data.csv", index=False)
print("Encoded dataset saved")


Encoded dataset saved


In [58]:
import pandas as pd

# Load both datasets
cleaned_df = pd.read_csv("cleaned_data.csv")
encoded_df = pd.read_csv("encoded_data.csv")

# Check row counts
if cleaned_df.shape[0] == encoded_df.shape[0]:
    print("✅ Row count matches.")
else:
    print("❌ Row count mismatch!")

# Check index match
if (cleaned_df.index == encoded_df.index).all():
    print("✅ Index match confirmed.")
else:
    print("ℹ️ Index mismatch. Resetting index...")
    cleaned_df.reset_index(drop=True, inplace=True)
    encoded_df.reset_index(drop=True, inplace=True)
    print("✅ Index reset and matched.")

✅ Row count matches.
✅ Index match confirmed.


In [59]:
from sklearn.cluster import KMeans

encoded_df = pd.read_csv("encoded_data.csv")

X = encoded_df.select_dtypes(include=['number'])

Kmeans = KMeans(n_clusters=4, random_state=42)

Kmeans.fit_predict(X)

encoded_df["Cluster"] = Kmeans.labels_

print(encoded_df.head())

   city_cleaned_Abohar  city_cleaned_Adilabad  city_cleaned_Adityapur  \
0                  1.0                    0.0                     0.0   
1                  1.0                    0.0                     0.0   
2                  1.0                    0.0                     0.0   
3                  1.0                    0.0                     0.0   
4                  1.0                    0.0                     0.0   

   city_cleaned_Adoni  city_cleaned_Agartala  city_cleaned_Agra  \
0                 0.0                    0.0                0.0   
1                 0.0                    0.0                0.0   
2                 0.0                    0.0                0.0   
3                 0.0                    0.0                0.0   
4                 0.0                    0.0                0.0   

   city_cleaned_Ahmedabad  city_cleaned_Ahmednagar  city_cleaned_Ajmer  \
0                     0.0                      0.0                 0.0   
1         

In [60]:
# Ensure indices match
cleaned_df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Add cluster info to cleaned_df
cleaned_df['Cluster'] = encoded_df['Cluster']

# Preview result
print(cleaned_df[['name', 'city_cleaned', 'cuisine', 'Cluster']].head())

                name city_cleaned        cuisine  Cluster
0  Janta Sweet House       Abohar  Sweets,Bakery        3
1  theka coffee desi       Abohar      Beverages        1
2   shere punjab veg       Abohar   North Indian        1
3  Bharawan Da Dhaba       Abohar         Indian        0
4    NIKKU VEG THALI       Abohar   North Indian        2


In [62]:
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(Kmeans, f)

In [61]:
with open("encoders.pkl", "wb") as f:
    pickle.dump({
        "Encoder": encoder, 
        "Label": label, 
        "KMeans": Kmeans
    }, f)